<a href="https://colab.research.google.com/github/be325/be325/blob/main/GRS35306_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# Upload file
uploaded = files.upload()

Saving waterdepth.xlsx to waterdepth.xlsx


In [ ]:
pip install meteostat

In [ ]:
# Import Meteostat library and dependencies
import pandas as pd
import numpy as np
from datetime import datetime
from meteostat import Point, Daily, Hourly
from meteostat import Stations
from meteostat import Hourly


from meteostat import Stations

start = datetime(2024, 12, 20)
end = datetime(2025, 1, 21)

x = 51.900652
y= 5.455519
#Determination of Stations for specific region
stations = Stations()
stations = stations.nearby(x, y) # Meteostat Station of NOAA and NCEI
station1 = stations.fetch(5)

#Data interval
data1 = Hourly(station1, start, end)
#data = data.normalize()
#data = data.aggregate('1W')
data1 = data1.interpolate()
data1 = data1.fetch()

#PREPARATION OF DATA SET

#Convert to Pandans.Core for Raw Data Set
station= pd.DataFrame(station1)
data= pd.DataFrame(data1)

#There is multi index in pandas. It should be one index. Otherwise, we can encounter some problems in ML part.
dataCleanStep1 = data.reset_index()


#We should pay attention NAN value and unuseful parameters. All of them should be removed.

dataCleanStep2 = pd.DataFrame(dataCleanStep1.drop(["snow","wpgt","tsun","coco"], axis =1))

cleanClimateData = dataCleanStep2.dropna(axis =0)

#Also you can there is NAN value with below code:
#CleanClimateData.isnull.values.any()
#Result should False. Otherwise, it is not joke :)

#########################################Prepearing for Water Depth###########################################

waterDepthdata1 = pd.read_excel("waterdepth.xlsx")

# Zaman sütununu datetime formatına çevirme
waterDepthdata1['Tijd (NL tijd)'] = pd.to_datetime(waterDepthdata1['Tijd (NL tijd)'], format='%H:%M:%S', errors='coerce')

# Convert the time column to datetime format
cleanwaterDepthdata1 = hourly_data = waterDepthdata1[waterDepthdata1['Tijd (NL tijd)'].dt.minute == 0]

# Filter data to include only hourly values
cleanWaterDepthData = cleanwaterDepthdata1.dropna(axis =0)

#####
# Actually, the following explanation needs to be made. Normally, we can access the water depth data from the Rijkwaterstaat website at intervals of every 10-20 minutes. However, we can only obtain climate data on an hourly basis. Therefore, we match every data point that ends with "00". For example, the water depth measured at 00:00 and the climate data recorded at 00:00 are paired on the same row.

# You might think, "How can precipitation falling in Germany also correspond to water depth in the Netherlands?" However, the climate data we fetch is already "fetched" according to the region. So, there's no reason to be concerned.
merged_df = pd.merge(cleanWaterDepthData, cleanClimateData, left_on='Datum', right_on='time', how='inner')
merged_df_ml = merged_df.drop(["Datum", "Locatie", "station","time", "Tijd (NL tijd)"], axis =1)

y = merged_df_ml["wd"].values
x= merged_df_ml.drop("wd", axis =1).values


print(x)
print(y)



from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=10)


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(15,activation="relu"))
model.add(Dense(15,activation="relu"))
model.add(Dense(15,activation="relu"))
model.add(Dense(15,activation="relu"))
model.add(Dense(15,activation="relu"))
model.add(Dense(15,activation="relu"))
model.add(Dense(15,activation="relu"))
model.add(Dense(15,activation="relu"))


model.add(Dense(1))

model.compile(optimizer="adam", loss="mse")

model.fit(x = x_train, y = y_train, validation_data=(x_test,y_test), batch_size=20, epochs =200)



from sklearn.metrics import mean_squared_log_error, mean_absolute_error

EstimationSeries = model.predict(x_test)
print(f"MAE: {mean_absolute_error(y_test,EstimationSeries)}")
print(f"MSE: {mean_squared_log_error(y_test,EstimationSeries)}")


from sklearn.metrics import r2_score
print(f"R-square: {r2_score(y_test,EstimationSeries)}")

################################################################
#Time to estimation using our ML model.
#Taking future climate data
from datetime import datetime, timedelta
startFuture = datetime.now()
endFuture = (datetime.now() + timedelta(hours=60))

x = 51.900652
y= 5.455519
#Determination of Stations for specific region
stations = Stations()
stations = stations.nearby(x, y) # Meteostat Station of NOAA and NCEI
station1 = stations.fetch(1)

#Data interval
dataFuture = Hourly(station1, startFuture, endFuture)
#data = data.normalize()
#data = data.aggregate('1W')
dataFuture = dataFuture.interpolate()
dataFuture = dataFuture.fetch()


futureCleandataStep1 = pd.DataFrame(dataFuture.drop(["snow","wpgt","tsun","coco"], axis =1))

futureCleanData = futureCleandataStep1.dropna(axis =0)


#Transform to array type
predictionData= scaler.transform(futureCleanData.values.reshape(-1,7))
waterDepthEstimation = pd.DataFrame(model.predict(predictionData))

############

waterDepthEstimation.index= futureCleanData.index
FutureEstimation = pd.concat([waterDepthEstimation, futureCleanData], axis=1)



[[ 4.5000e+00  2.0000e+00  8.4000e+01 ...  2.6900e+02  1.8000e+01
   1.0137e+03]
 [ 5.6000e+00  1.9000e+00  7.7000e+01 ...  2.8000e+02  1.0800e+01
   1.0132e+03]
 [ 4.0000e+00  2.8000e+00  9.2000e+01 ...  2.5000e+02  1.4400e+01
   1.0135e+03]
 ...
 [ 3.0000e-01  2.0000e-01  9.9000e+01 ...  1.6200e+02  7.4000e+00
   1.0371e+03]
 [-2.0000e-01 -3.0000e-01  9.9000e+01 ...  1.1900e+02  7.0000e+00
   1.0379e+03]
 [ 0.0000e+00  0.0000e+00  1.0000e+02 ...  1.4000e+02  4.0000e+00
   1.0370e+03]]
[305. 305. 305. ... 410. 410. 410.]
Epoch 1/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 146766.2188 - val_loss: 103230.5234
Epoch 2/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 44248.8906 - val_loss: 5619.9150
Epoch 3/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5784.2856 - val_loss: 4898.4697
Epoch 4/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4959.5269 - val_loss: 4479.6909
Epoch 5/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4467.6758 - val_loss: 4111.1104
Ep

In [ ]:
import plotly.express as px
fig_data = FutureEstimation.reset_index()
publicPredictionLine= px.line(fig_data, x ="time", y="dwpt", title='Future Water Depth in the Netherlands')

In [ ]:
publicPredictionLine

In [ ]:
datacoor = [
    [51.658439, 6.554736],
    [51.879550, 6.032289],
    [51.819524, 6.266566],
    [51.880071, 5.756830],
    [51.869088, 5.896421],
    [51.993134, 6.055172],
    [51.977824, 5.874026],
    [51.959651, 5.779345],
    [51.941022, 5.630768],
    [51.964363, 5.545555],
    [51.982085, 5.477094],
    [51.981861, 5.426476],
    [51.977600, 5.230194],
    [51.987692, 5.060497],
    [51.963914, 4.942873]
]



dataMAP = pd.DataFrame(datacoor, columns=['Latitude', 'Longitude'])


from datetime import datetime, timedelta
startFuture = datetime.now()
endFuture = (datetime.now() + timedelta(hours=20))

x = 51.900652
y= 5.455519
#Determination of Stations for specific region
stationsm = Stations()
stationsm = stationsm.nearby(x, y) # Meteostat Station of NOAA and NCEI
station1m = stationsm.fetch(1)

#Data interval
dataFuturem = Hourly(station1m, startFuture, endFuture)
#data = data.normalize()
#data = data.aggregate('1W')
dataFuturem = dataFuturem.interpolate()
dataFuturem = dataFuturem.fetch()


futureCleandataStep1m = pd.DataFrame(dataFuturem.drop(["snow","wpgt","tsun","coco"], axis =1))

futureCleanDatam = futureCleandataStep1m.dropna(axis =0)


#Transform to array type
predictionDatam= scaler.transform(futureCleanDatam.values.reshape(-1,7))
waterDepthEstimationm = pd.DataFrame(model.predict(predictionDatam), columns=["WaterDepth"])

dataWaterdepthandcoor = pd.merge(dataMAP, waterDepthEstimationm, left_index=True, right_index=True, suffixes=('dataMAP', 'waterDepthEstimationm'))


###MAP

figPublic = px.scatter_mapbox(dataWaterdepthandcoor,
                        lat=dataWaterdepthandcoor["Latitude"],
                        lon=dataWaterdepthandcoor["Longitude"],
                        color = "WaterDepth",
                        #hover_name=dataFrameStationTemperature["name"],
                        #hover_data=["ETo_current", "ETo_selected"],
                        size = "WaterDepth",
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        size_max = 50,
                        zoom=4.1,
                        width =1200,
                        height=500,
                        mapbox_style = "carto-positron"
                        )



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step


In [ ]:
figPublic

In [ ]:
dataDescriptionStep1 = merged_df.describe()
dataDescription = dataDescriptionStep1.drop(["Datum", "Tijd (NL tijd)"], axis=1)
print(dataDescription)

                wd                           time         temp         dwpt  \
count  2488.000000                           2488  2488.000000  2488.000000   
mean    379.106109  2025-01-03 10:29:42.636655872     3.404502     2.506190   
min     303.000000            2024-12-20 00:00:00    -3.200000    -5.000000   
25%     327.000000            2024-12-28 00:00:00     1.100000     0.200000   
50%     353.500000            2025-01-04 00:00:00     3.200000     2.100000   
75%     430.000000            2025-01-10 00:00:00     5.600000     4.825000   
max     518.000000            2025-01-17 00:00:00    11.000000     9.800000   
std      64.021797                            NaN     3.298592     3.265697   

              rhum         prcp         wdir         wspd         pres  
count  2488.000000  2488.000000  2488.000000  2488.000000  2488.000000  
mean     93.948151     0.082034   208.942122    13.016399  1019.900121  
min      77.000000     0.000000     0.000000     0.000000   986.50000

In [ ]:
privatePredictionLine= px.line(dataWaterdepthandcoor, y="WaterDepth", title='Future Water Depth in the Netherlands')

In [ ]:
privatePredictionLine

In [ ]:
ModelPerformance= px.line(costline,x="val_loss", y="loss", title='Model Performance')

NameError: name 'costline' is not defined

In [ ]:
ModelPerformance

NameError: name 'ModelPerformance' is not defined

In [ ]:
merged_df_ml

,wd,temp,dwpt,rhum,prcp,wdir,wspd,pres
0,305.0,4.5,2.0,84.0,0.0,269.0,18.0,1013.7
1,305.0,5.6,1.9,77.0,0.0,280.0,10.8,1013.2
2,305.0,4.0,2.8,92.0,0.3,250.0,14.4,1013.5
3,305.0,3.1,2.2,94.0,0.0,254.0,14.4,1012.5
4,305.0,4.5,2.0,84.0,0.0,269.0,18.0,1013.7
...,...,...,...,...,...,...,...,...
2483,412.0,0.0,0.0,100.0,0.0,140.0,4.0,1037.0
2484,410.0,-0.8,-1.4,96.0,0.0,173.0,5.4,1037.6
2485,410.0,0.3,0.2,99.0,0.0,162.0,7.4,1037.1
2486,410.0,-0.2,-0.3,99.0,0.0,119.0,7.0,1037.9


In [ ]:
import plotly.express as px

fig1 = px.line(merged_df_ml, y=["wd"])
fig2 = px.line(merged_df_ml, y=["temp"])
fig3 = px.line(merged_df_ml, y=["rhum"])
fig4 = px.line(merged_df_ml, y=["prcp"])
fig5 = px.line(merged_df_ml, y=["wdir"])
fig6 = px.line(merged_df_ml, y=["pres"])
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()
fig6.show()

